Negotiation as Search Replication Code Step 2

# Step 2: Agent-based simulation


In [ ]:
# Install and import python-igraph safely (no manual runtime restart needed)
import os
import sys
import glob
import time
import random
import pathlib
import datetime
import itertools
import warnings
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm


from tqdm import tqdm

def ensure_python_igraph():
    try:
        import igraph as ig  # already installed and importable
        return ig
    except Exception:
        pass

    # Install only if not importable yet
    try:
        # python-igraph is the correct package name on PyPI
        # cairocffi enables plotting without apt-get
        !python -m pip install -q python-igraph cairocffi
    except Exception as e:
        print("pip install failed:", e)

    # Try import again
    try:
        import igraph as ig
        return ig
    except Exception as e:
        # As a fallback, try pycairo if cairocffi was not enough
        try:
            !python -m pip install -q pycairo
            import igraph as ig
            return ig
        except Exception as e2:
            raise ImportError(
                "igraph could not be imported after install. "
                "If this is a reused session that already imported an old igraph, "
                "use Runtime -> Restart runtime once and re-run this cell first."
            ) from e2

ig = ensure_python_igraph()
print("igraph version:", ig.__version__)

# Optional: quick self-test (no plotting)
g = ig.Graph.Erdos_Renyi(n=5, m=6)
print("Test graph:", g.summary())

# Step 2 setup: consistent folders for reading Step 1 outputs and writing Step 2 outputs
import os
from pathlib import Path

def in_colab():
    return "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ

PROJECT_ROOT = Path.cwd()

if in_colab():
    try:
        from google.colab import drive  # type: ignore
        drive.mount("/content/drive", force_remount=False)
        drive_proj = Path("/content/drive/MyDrive/Negotiations-as-Search")
        if drive_proj.exists():
            PROJECT_ROOT = drive_proj
    except Exception as e:
        print("Drive not mounted:", e)

# Constant folders (your chosen naming)
S1_OUTPUT = PROJECT_ROOT / "S1_OUTPUT"   # inputs for Step 2
S2_OUTPUT = PROJECT_ROOT / "S2_OUTPUT"   # outputs from Step 2
S1_OUTPUT.mkdir(parents=True, exist_ok=True)
S2_OUTPUT.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJECT_ROOT)
print("Step 1 outputs (read from):", S1_OUTPUT)
print("Step 2 outputs (write to): ", S2_OUTPUT)

# Short helpers
def s1out(name): return S1_OUTPUT / name
def s2out(name): return S2_OUTPUT / name


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
igraph version: 1.0.0
Test graph: IGRAPH U--- 5 6 -- 
Mounted at /content/drive
Project root: /content/drive/MyDrive/Negotiations-as-Search
Step 1 outputs (read from): /content/drive/MyDrive/Negotiations-as-Search/S1_OUTPUT
Step 2 outputs (write to):  /content/drive/MyDrive/Negotiations-as-Search/S2_OUTPUT


### Simulation Helper Functions

In [ ]:
# -------------------------------------------------------------------
#  Transition‑rule filter  (2025‑05‑16 final)
# -------------------------------------------------------------------
def apply_transition_rules_to_dataframe(df: pd.DataFrame,
                                        rule_number: int) -> pd.DataFrame:
    def _none(series):
        """True for NA, None or the string 'None'."""
        return series.isna() | (series == "None")

    if rule_number == 1:                  # vote‑gain only
        cond = df["Vote_Change"] > 0

    elif rule_number == 2:                # tie‑resolution
        # keep all ties; keep any edge with a positive vote gain
        cond = (df["Vote_Change"] > 0) | (df["Vote_Change"] == 0)

    elif rule_number == 3:                # proposer‑accept
        # positive votes must also satisfy a non‑empty Accepting list
        cond = ((df["Vote_Change"] > 0) & ~_none(df["Accepting"])) | \
               (df["Vote_Change"] == 0)

    elif rule_number == 4:                # Pareto
        cond = (df["Vote_Change"] > 0) | (df["Vote_Change"] == 0)

    else:                                 # joint‑payoff (legacy)
        cond = (df["Vote_Change"] > 0) | \
               ((df["Vote_Change"] == 0) & (df["Payoff_Change"] > 0))

    return df[cond].copy()

### Extracting party names (used in transition rule code1)
def extract_parties(df):
    """
    Extracts the list of parties from the DataFrame columns based on the pattern [party name]_destination_payoff.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing negotiation data.

    Returns:
    list: A list of unique party names.
    """
    party_columns = [col for col in df.columns if '_destination_payoff' in col]
    parties = [col.replace('_destination_payoff', '') for col in party_columns]
    return list(set(parties))


def set_agreement_threshold(nego_df, N, rule_choice):
    """
    Sets the agreement threshold based on the specified rule.

    Parameters:
    nego_df (DataFrame): The negotiation Data.Frame.
    N (int): Number of parties.
    rule_choice (int): The choice of rule.
                       1 for simple majority threshold,
                       2 for max votes minus one.

    Returns:
    DataFrame: The updated negotiation DataFrame with new agreement columns.
    """
    if rule_choice == 1:
        # Majority threshold ule
        majority_threshold = N // 2
        # nego_df['Origin_Agreement'] = ((nego_df['Origin_votes'] > majority_threshold) & (nego_df['Origin_veto'] == 'No')).astype(int)
        nego_df['Origin_Agreement'] = ((nego_df['Origin_votes'] > majority_threshold)).astype(int)
        # nego_df['Destination_Agreement'] = ((nego_df['Destination_votes'] > majority_threshold) & (nego_df['Destination_veto'] == 'No')).astype(int)
        nego_df['Destination_Agreement'] = ((nego_df['Destination_votes'] > majority_threshold)).astype(int)
    elif rule_choice == 2:
        # Max votes minus one rule
        agreement_threshold = max(nego_df['Origin_votes']) - 1
        nego_df['Origin_Agreement'] = ((nego_df['Origin_votes'] >= agreement_threshold) & (nego_df['Origin_veto'] == 'No')).astype(int)
        nego_df['Destination_Agreement'] = ((nego_df['Destination_votes'] >= agreement_threshold) & (nego_df['Destination_veto'] == 'No')).astype(int)
    elif rule_choice == 3:
        # Unanimous vote agreement rule
        unanimous_threshold = N  # Agreement threshold is equal to the number of parties
        nego_df['Origin_Agreement'] = (nego_df['Origin_votes'] == unanimous_threshold).astype(int)
        nego_df['Destination_Agreement'] = (nego_df['Destination_votes'] == unanimous_threshold).astype(int)
    else:
        raise ValueError("Invalid rule choice. Please select 1, 2, or 3.")

    return nego_df

### Extracting party names (used in transition rule code1)
def extract_parties(df):
    """
    Extracts the list of parties from the DataFrame columns based on the pattern [party name]_destination_payoff.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing negotiation data.

    Returns:
    list: A list of unique party names.
    """
    party_columns = [col for col in df.columns if '_destination_payoff' in col]
    parties = [col.replace('_destination_payoff', '') for col in party_columns]
    return list(set(parties))

### TRANSITION RULE
def apply_transition_rules_to_dataframe(df, rule_number):
    """
    Applies specified transition rules to the DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame on which to apply the transition rules.
    rule_number (int): The number of the transition rule to apply.

    Rules:
    1. Keep rows if 'Vote_Change' is strictly positive.
    2. Keep rows if 'Vote_Change' is positive. If 'Vote_Change' is zero, keep if 'Payoff_Change' (inc. in joint pay-offs) is positive.
    3. Keep rows if 'Vote_Change' is positive. If 'Vote_Change' is zero, keep if 'Worse_Off' is 'None'.
    4. (Concessions Rule) Randomly selects one party and transitions to the node where this party's payoff is the highest.
    """

    # Ensure the rule_number is valid
    if rule_number not in [1, 2, 3, 4]:
        raise ValueError("Invalid rule number. Please select 1, 2, 3, or 4.")

    # Define the conditions for keeping rows based on the rule
    if rule_number == 1:
        # Strictly increase in votes
        condition = (df['Vote_Change'] > 0)
    elif rule_number == 2:
        # Increase in total vote count or if votes are equal, then increase in joint payoff
        condition = (df['Vote_Change'] > 0) | ((df['Vote_Change'] == 0) & (df['Payoff_Change'] > 0))
    elif rule_number == 3:
        # Increase in total vote count or if votes are equal, then pareto Superiority applies (consider outcomes where no one should be worse off)
        condition = (df['Vote_Change'] > 0) | ((df['Vote_Change'] == 0) & (df['Worse_Off'] == 'None'))
    elif rule_number == 4:
        # Concessions Rule: Transitions based on maximizing the payoff for a randomly selected party.
        parties = extract_parties(df)
        selected_party = random.choice(parties)
        payoff_column = f"{selected_party}_destination_payoff"
        df['Selected_Party_Payoff'] = df[payoff_column]
        max_payoff = df['Selected_Party_Payoff'].max()
        condition = df['Selected_Party_Payoff'] == max_payoff
    else:
        raise ValueError("Unexpected rule number.")

    # Filter the DataFrame based on the condition
    filtered_df = df[condition].copy()

    # Clean up the temporary column if used
    if 'Selected_Party_Payoff' in filtered_df.columns:
        del filtered_df['Selected_Party_Payoff']

    return filtered_df

import math
import random
import pandas as pd
from tqdm import tqdm

###############################################################################
# Utility functions for proposal extraction from the negotiation DataFrame.
###############################################################################

def hamming_distance(s1, s2):
    """Compute the Hamming distance between two strings (assumes equal length)."""
    s1, s2 = str(s1), str(s2)
    if len(s1) != len(s2):
        return max(len(s1), len(s2))
    return sum(ch1 != ch2 for ch1, ch2 in zip(s1, s2))

def get_filler_offers_for_anchor(anchor, nego_df, fill_search_distance=1):
    """
    Given an anchoring offer (anchor), return a list of potential filler offers from nego_df's Destination column.
    Only offers with Hamming distance <= fill_search_distance (and not equal to anchor) are returned.
    """
    candidates = nego_df['Destination'].unique().tolist()
    fillers = [offer for offer in candidates if offer != anchor and hamming_distance(offer, anchor) <= fill_search_distance]
    return list(set(fillers))  # ensure uniqueness

def _get_dest_payoffs(offer, df, parties):
    row = df.loc[df["Destination"] == offer]
    if row.empty:
        return {p: None for p in parties}
    row = row.iloc[0]
    return {p: row.get(f"{p}_destination_payoff", None) for p in parties}

def _attach_final_payoffs(runs, nego_df, rp_df):
    parties = rp_df["Party"].tolist()
    rp_map  = rp_df.set_index("Party")["Reservation"].to_dict()
    for p in parties:
        runs[f"{p}_Final_Payoff"] = [
            _get_dest_payoffs(r.end_node, nego_df, parties).get(p, 0)
            if r.agreement and r.end_node is not None else rp_map.get(p, 0)
            for _, r in runs.iterrows()
        ]

def extract_parties_max_payoff(df, top_value=3, top_percent=True, random_seed=None):
    """
    Extract the top payoff proposals for each party from the provided DataFrame.

    Parameters:
      df (DataFrame): The negotiation DataFrame. It should include columns named like "<party>_origin_payoff"
                      and an "Origin" column with the proposal IDs.
      top_value (int or float, default=3):
          - If top_percent is False, top_value is the number of proposals to keep.
          - If top_percent is True, top_value (0 < top_value <= 100) is interpreted as a percentage.
      top_percent (bool, default=True):
          - If True, returns the top (top_value)% of proposals for each party.
          - If False, returns the top top_value proposals.
      random_seed (int, optional): A seed for reproducible random tie-breaks.

    Returns:
      dict: Mapping each party (inferred from the payoff column names) to a list of proposals in the top segment.
    """
    if random_seed is not None:
        random.seed(random_seed)

    parties_top_payoff = {}
    for col in df.columns:
        if '_origin_payoff' in col:
            party_name = col.replace('_origin_payoff', '')
            party_df = df[['Origin', col]].dropna()
            party_df = party_df.drop_duplicates(subset='Origin')
            party_df = party_df.sort_values(by=col, ascending=False).reset_index(drop=True)
            # Determine how many proposals to take.
            if top_percent:
                n_total = len(party_df)
                n_take = max(1, math.ceil(n_total * top_value / 100))
            else:
                n_take = int(top_value)
            # Handle ties at the boundary.
            if n_take < len(party_df):
                boundary_value = party_df.iloc[n_take - 1][col]
                count_above = (party_df[col] > boundary_value).sum()
                spots = n_take - count_above
                tied_rows = party_df[party_df[col] == boundary_value]
                selected_rows = party_df[party_df[col] > boundary_value]
                selected_proposals = selected_rows['Origin'].tolist()
                tied_proposals = tied_rows['Origin'].tolist()
                if spots < len(tied_proposals):
                    selected_tied = random.sample(tied_proposals, spots)
                else:
                    selected_tied = tied_proposals
                result_proposals = selected_proposals + selected_tied
            else:
                result_proposals = party_df.head(n_take)['Origin'].tolist()
            parties_top_payoff[party_name] = result_proposals
    return parties_top_payoff

def create_payoff_reports_with_votes(df, parties_top_payoff, filtered_df_hamming=None,
                                     top_value=3, top_percent=True, random_seed=None):
    """
    Create a payoff report with votes using the given parties_top_payoff dictionary.

    Parameters:
      df (DataFrame): The negotiation DataFrame.
      parties_top_payoff (dict): Mapping from parties to a list of proposals.
      filtered_df_hamming (DataFrame, optional): Used for additional checking.
      top_value (int or float, default=3) and top_percent (bool, default=True): Same as in extract_parties_max_payoff.
      random_seed (int, optional): For reproducible tie-breaks.

    Returns:
      DataFrame: A report with columns for Party, Proposal, Payoff, and Votes.
    """
    payoff_vote_info = []
    for party, proposals in parties_top_payoff.items():
        for proposal in proposals:
            payoff_column = f'{party}_origin_payoff'
            matched_rows = df[df['Origin'] == proposal]
            if not matched_rows.empty:
                payoff_value = matched_rows[payoff_column].iloc[0]
                votes = matched_rows['Origin_votes'].iloc[0]
                payoff_vote_info.append({
                    'Party': party,
                    'Proposal': proposal,
                    'Payoff': payoff_value,
                    'Votes': votes
                })
    payoff_votes_df = pd.DataFrame(payoff_vote_info)
    if filtered_df_hamming is not None and not payoff_votes_df.empty:
        in_filtered = payoff_votes_df['Proposal'].isin(filtered_df_hamming['Origin']) | \
                      payoff_votes_df['Proposal'].isin(filtered_df_hamming['Destination'])
        if not in_filtered.all():
            print("Some proposals in the payoff report were not found in the filtered DataFrame. Rebuilding...")
            fallback_parties = extract_parties_max_payoff(filtered_df_hamming,
                                                          top_value=top_value,
                                                          top_percent=top_percent,
                                                          random_seed=random_seed)
            return create_payoff_reports_with_votes(filtered_df_hamming, fallback_parties,
                                                    filtered_df_hamming=filtered_df_hamming,
                                                    top_value=top_value,
                                                    top_percent=top_percent,
                                                    random_seed=random_seed)
    return payoff_votes_df

def _hamming(a: str, b: str) -> int:
    a, b = str(a), str(b)
    if len(a) != len(b):
        w = max(len(a), len(b))
        a, b = a.zfill(w), b.zfill(w)
    return sum(x != y for x, y in zip(a, b))

def generate_chosen_solution_info(
        nego_df: pd.DataFrame,
        *,
        filtered_df_hamming: pd.DataFrame | None = None,
        first_offer_rule: str = "top_percent",
        top_value: int = 3,
        top_percent: bool = True,
        random_seed: int | None = None
) -> pd.DataFrame:
    """
    Return the starting-proposal table used by the simulator.

    •  first_offer_rule  ∈ {"top_percent", "top_value", "random", "meso"}
       – "meso" performs a pre-vote among each party’s best
         `top_value` proposals and keeps the one with the highest Origin_votes.

    •  top_value is the candidate pool size.
       The variable is still threaded through main_search_model as *tv*.
    """
    if random_seed is not None:
        random.seed(random_seed)

    if first_offer_rule.lower() in {"top_percent", "top_value"}:
        parties_top = extract_parties_max_payoff(
            nego_df,
            top_value=top_value,
            top_percent=(first_offer_rule.lower() == "top_percent"),
            random_seed=random_seed
        )
        chosen_df = create_payoff_reports_with_votes(
            nego_df,
            parties_top,
            filtered_df_hamming=filtered_df_hamming,
            top_value=top_value,
            top_percent=(first_offer_rule.lower() == "top_percent"),
            random_seed=random_seed
        )

    elif first_offer_rule.lower() == "random":
        parties = [c.replace("_origin_payoff", "")
                   for c in nego_df.columns if "_origin_payoff" in c]
        records = []
        for party in parties:
            rows = nego_df[["Origin", f"{party}_origin_payoff", "Origin_votes"]] \
                     .dropna().drop_duplicates(subset="Origin")
            if not rows.empty:
                pick = rows.sample(n=1, random_state=random_seed)
                records.append(
                    {"Party": party,
                     "Proposal":  pick["Origin"].iat[0],
                     "Payoff":    pick[f"{party}_origin_payoff"].iat[0],
                     "Votes":     pick["Origin_votes"].iat[0]}
                )
        chosen_df = pd.DataFrame(records)

    elif first_offer_rule.lower() == "meso":
        # 1.  grab each party’s best `top_value` payoff offers
        parties_top = extract_parties_max_payoff(
            nego_df,
            top_value=top_value,
            top_percent=False,
            random_seed=random_seed
        )

        # 2.  within that set choose the one with the most votes
        records = []
        for party, proposals in parties_top.items():
            best_votes, best_prop, best_payoff = -1, None, None
            for prop in proposals:
                row = nego_df.loc[nego_df["Origin"] == prop].iloc[0]
                votes  = row["Origin_votes"]
                payoff = row[f"{party}_origin_payoff"]
                if votes > best_votes:
                    best_votes, best_prop, best_payoff = votes, prop, payoff
                elif votes == best_votes:          # tie → random
                    if random.random() < 0.5:
                        best_prop, best_payoff = prop, payoff
            records.append(
                {"Party": party,
                 "Proposal": best_prop,
                 "Payoff":   best_payoff,
                 "Votes":    best_votes}
            )
        chosen_df = pd.DataFrame(records)

    else:
        raise ValueError("first_offer_rule must be one of "
                         "'top_percent', 'top_value', 'random', 'meso'.")

    return chosen_df



def reservation_callback(csv_name: str, parties: list[str], df_raw: pd.DataFrame) -> pd.DataFrame:
    """
    Return a DataFrame with columns [Party, Reservation] for *this* csv_name.
    Edit the mapping below as you like; fallback = 0 for all parties.
    """
    preset: dict[str, dict[str, list]] = {
        "harborco_transition_analysis_transition_analysis.csv" : {
            'Party': ['EL', 'FD', 'G', 'H', 'OP', 'U'],
            'Reservation': [50, 65, 30, 55, 31, 50]
        },
        "FL_old_transition_analysis.csv" : {
            'Party': ['SC', 'GLC', 'TIG', 'MG', 'OB', 'NPC'],
            'Reservation': [45, 47, 55, 40, 47, 65]
        },
        "FL_new_transition_analysis.csv" : {
            'Party': ['SC', 'GLC', 'TIG', 'MG', 'OB', 'NPC'],
            'Reservation': [32, 51, 55, 40, 40, 40]
        },
        "new_recruit_mod_transition_analysis_transition_analysis.csv" : {
            'Party': ['R', 'C'],
            'Reservation': [4500, 4500]
        },
    }

    if csv_name in preset:
        return pd.DataFrame(preset[csv_name])

    # default – zeros
    return pd.DataFrame({"Party": parties, "Reservation": [0] * len(parties)})


 #───────────────────────── minor helpers ────────────────────────────────────
def _hamming(a: str, b: str) -> int:
    a, b = str(a), str(b)
    if len(a) != len(b):
        w = max(len(a), len(b))
        a, b = a.zfill(w), b.zfill(w)
    return sum(x != y for x, y in zip(a, b))

def compute_possible_solutions(transition_df):
    """
    Count every unique offer that exists in the raw transition CSV.
    Offers appear in the string columns 'Origin' and 'Destination'.
    """
    # collect both endpoints of every transition, drop duplicates, count
    all_offers = pd.concat([
        transition_df["Origin"],
        transition_df["Destination"]
    ])
    return all_offers.drop_duplicates().shape[0]

def compute_exploration_rate(offers_seen, possible_solutions):
    """
    Fraction of the landscape visited IN ONE RUN.
    `offers_seen` is your list/Series of offers proposed this run.
    """
    return len(set(offers_seen)) / possible_solutions


def _get_dest_payoffs(dest, df, parties):
    row = df.loc[df["Destination"] == dest]
    if row.empty:
        return {p: None for p in parties}
    row = row.iloc[0]
    return {p: row.get(f"{p}_destination_payoff", None) for p in parties}


def _attach_final_payoffs(df_runs, nego_df, reservation_df):
    parties = reservation_df["Party"].tolist()
    res_map = reservation_df.set_index("Party")["Reservation"].to_dict()
    for p in parties:
        df_runs[f"{p}_Final_Payoff"] = [
            _get_dest_payoffs(r.end_node, nego_df, parties).get(p, 0)
            if r.agreement == 1 and r.end_node else res_map.get(p, 0)
            for _, r in df_runs.iterrows()
        ]

## new helpers
# === Helpers: csv pruning, scenario typing, unique codes, robust input read ===
import os, re, json, hashlib
from pathlib import Path

GENERIC_RE = re.compile(r"UM_N=\d+_M=\d+_", re.I)

def sanitize_csv_source(path_str: str) -> str:
    """Return just the basename 'file.csv' from any path."""
    return os.path.basename(str(path_str))

def scenario_type_from_basename(csv_base: str) -> str:
    """Classify scenario as 'RanGen' if it matches the UM_N=..._M=..._ pattern, else 'Manual'."""
    return "RanGen" if GENERIC_RE.search(csv_base) else "Manual"

def ensure_unique_key(store: dict, base_key: str) -> str:
    """If base_key exists in store, append __2, __3, ... until unique."""
    if base_key not in store:
        return base_key
    i = 2
    while f"{base_key}__{i}" in store:
        i += 1
    return f"{base_key}__{i}"

def make_simulation_code(csv_base: str, *, a0, da, b0, db, g0, dg, t0, dt, dmax, agr, trn, fo) -> str:
    """
    Informative + stable short hash. Example:
      FL_old_transition_analysis__tr1_S1_fo_meso_a0.5_b0_g0_t0__3a1c9b72
    """
    core = {
        "csv": csv_base, "a0": a0, "da": da, "b0": b0, "db": db,
        "g0": g0, "dg": dg, "t0": t0, "dt": dt,
        "S": dmax, "agr": agr, "tr": trn, "fo": fo
    }
    short = hashlib.sha1(json.dumps(core, sort_keys=True).encode("utf-8")).hexdigest()[:8]
    stem = Path(csv_base).stem
    return f"{stem}__tr{trn}_S{dmax}_fo_{fo}_a{a0}_b{b0}_g{g0}_t{t0}__{short}"

def read_input_csv(csv_name: str, output_path) -> tuple[pd.DataFrame, str]:
    """
    Robust reader:
      - If csv_name is an existing path (absolute or relative), read as-is.
      - Else try output_path / csv_name (legacy behavior).
      - Else raise with a helpful error.
    Returns (df_raw, resolved_path_str).
    """
    p = Path(csv_name)
    if p.exists():
        return pd.read_csv(p), str(p)
    p2 = Path(output_path) / csv_name
    if p2.exists():
        return pd.read_csv(p2), str(p2)
    raise FileNotFoundError(
        f"Could not locate input CSV '{csv_name}'. Tried as-is and under '{output_path}'."
    )

def print_simulation_plan(csvs_resolved: list[str], grids: dict):
    """Concise, organized run plan."""
    bases = [sanitize_csv_source(c) for c in csvs_resolved]
    a_vals  = sorted(set(map(float, grids["a0"])))
    tr_vals = sorted(set(map(int,   grids["trn"])))
    fo_vals = list(dict.fromkeys(grids["fo"]))  # keep order, unique
    S_vals  = sorted(set(map(int,   grids["dmax"])))

    print("\n=== Simulation Plan ===")
    print("CSV sources   :", ", ".join(bases))
    print("alpha         :", a_vals)
    print("transition_rule:", tr_vals)
    print("first_offer_rule:", fo_vals)
    print("max_search_distance (S):", S_vals)
    print("=======================\n")


# accessor ───────────────────────────────────────────────────────────────────
def get_simulation(code: str):
    return SIMULATION_STORE.get(code)


# === Step 2 auto-save helper ===
from pathlib import Path
import datetime
import pandas as pd

# Accessor
def get_simulation(code: str, store=None):
    if store is None:
        store = globals().get("SIMULATION_STORE", {})
    return store.get(code)

# Saver
def save_results(summary_df: pd.DataFrame,
                 dest_folder,
                 store=None) -> Path:
    """
    Save Step 2 outputs:
      - 0_summary.csv
      - all per-run CSVs from SIMULATION_STORE
    Creates a timestamped subfolder inside dest_folder.
    """
    if store is None:
        store = globals().get("SIMULATION_STORE", {})
    dest_folder = Path(dest_folder)
    dest_folder.mkdir(parents=True, exist_ok=True)

    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    out = dest_folder / f"simulation_{ts}"
    out.mkdir(parents=True, exist_ok=True)

    summary_df.to_csv(out / "0_summary.csv", index=False)
    for code, df in store.items():
        safe_code = "".join(c if c.isalnum() or c in "._-" else "_" for c in str(code))
        df.to_csv(out / f"{safe_code}.csv", index=False)

    print(f"Saved results to {out}")
    return out


### Main Simulation Code

In [ ]:
#V5───────────────────────── core engine (ONE param-combo) ────────────────────
SIMULATION_STORE: dict[str, pd.DataFrame] = {}

def _run_simulation(
        G, starts_df, consensus_nodes, nego_df, *,
        num_simulations,
        first_offer_rule,
        top_value,                 # candidate pool size
        alpha0, delta_alpha,
        beta0,  delta_beta,  beta_memory,
        gamma0, delta_gamma, gamma_dist,
        theta0, delta_theta, theta_override_alpha,
        include_filler_offers, max_search_distance,
        random_seed, add_impasse, apply_post_processing,
        record_full_histories,
        transition_rule
):
    """
    Execute *num_simulations* random walks on graph **G**.
    Now: initial placements and all teleports (θ/γ/αβ) count as agreements
    if they land on a consensus node. Also we record teleports into the path.
    """
    parties  = starts_df["Party"].unique()
    vote_max = nego_df.groupby("Destination")["Destination_votes"].max().to_dict()

    # ── SAFE LOOKUP HELPERS to avoid .iloc[0] crashes ───────────────────────
    def _edge_row(df, origin, dest):
        r = df[(df["Origin"] == origin) & (df["Destination"] == dest)]
        return None if r.empty else r.iloc[:1].squeeze()

    def _origin_row(df, origin):
        r = df[df["Origin"] == origin]
        if r.empty:
            r = df[df["Destination"] == origin]   # fallback if node only appears as Destination
        return None if r.empty else r.iloc[:1].squeeze()

    def _party_accepts(party: str, acc) -> bool:
        if pd.isna(acc) or acc == "None":
            return False
        return party in {x.strip() for x in str(acc).split(",")}

    # runs ───────────────────────────────────────────────────────────────────
    runs = []
    all_start_offers = list(G.vs["name"]) if first_offer_rule == "random" else None

    for run_idx in range(num_simulations):
        if random_seed is not None:
            random.seed(random_seed + run_idx)   # unique RNG stream per run
        visited_nodes = {p: set() for p in parties}

        α, β, γ, θ = alpha0, beta0, gamma0, theta0
        agents, phist, phist_val = {}, {}, {}
        path, ann, ann_val = [], [], []
        below_rp = {p: 0 for p in parties}
        reached = False
        cache_votes = {}
        first_offer = None

        # ---------- helper: record a placement/jump/move & check consensus ----------
        def _record_visit(party: str, node_name: str, from_name: str | None):
            """
            Append node_name to path and histories (no edge required),
            update visited set and cache_votes, then check consensus.
            Returns True if this visit reached consensus.
            """
            # mark visited (per party)
            visited_nodes[party].add(node_name)

            # tag safely using origin-row context (best-effort)
            row0 = _origin_row(nego_df, node_name)
            tagv_votes = row0.get("Origin_votes", None) if row0 is not None else None
            tagv_pay   = row0.get(f"{party}_origin_payoff", None) if row0 is not None else None

            tag  = f"{party}_{node_name}*"
            tagv = f"{tag} ({tagv_votes}, {tagv_pay})"

            path.append(node_name)
            ann.append(tag)
            ann_val.append(tagv)
            phist.setdefault(party, []).append(tag)
            phist_val.setdefault(party, []).append(tagv)

            # cache max votes for this node
            cache_votes.setdefault(node_name, vote_max.get(node_name, 0))

            # consensus check
            return (node_name in consensus_nodes)

        # ── first anchors (round-robin) ────────────────────────────────────
        init_order = list(parties); random.shuffle(init_order)
        for p in init_order:
            opts = (all_start_offers if first_offer_rule == "random"
                    else starts_df.loc[starts_df["Party"] == p, "Proposal"].tolist())
            if not opts:
                continue
            choice = random.choice(opts)
            if choice not in G.vs["name"]:
                continue
            node = G.vs.find(name=choice)
            if first_offer is None:
                first_offer = p
            agents[p] = {"current": node, "active": True}

            # record initial placement and trigger agreement if it's consensus
            if _record_visit(p, node["name"], from_name=None):
                reached = True
                agents[p]["active"] = False
                # early exit from initial placement if any party starts on consensus
                break

        # ── active loop ────────────────────────────────────────────────────
        while any(a["active"] for a in agents.values()) and not reached:
            active = [q for q,a in agents.items() if a["active"]]
            random.shuffle(active)

            for p in active:
                cur = agents[p]["current"]  # igraph vertex

                # θ random jump (teleport)
                if not reached and (random.random() < θ and (theta_override_alpha or random.random() >= α)):
                    dest = random.choice(G.vs["name"])
                    if dest != cur["name"]:
                        cur = G.vs.find(name=dest)
                        agents[p]["current"] = cur
                        # record teleport; if consensus, stop
                        if _record_visit(p, cur["name"], from_name=None):
                            reached = True
                            agents[p]["active"] = False
                            break

                # γ fuzzy anchor (teleport within Hamming radius)
                if not reached and (random.random() < γ):
                    pool = [d for d in nego_df["Destination"].unique()
                            if 0 < _hamming(d, cur["name"]) <= gamma_dist]
                    if pool:
                        fuzz = random.choice(pool)
                        if fuzz in G.vs["name"] and fuzz != cur["name"]:
                            cur = G.vs.find(name=fuzz)
                            agents[p]["current"] = cur
                            if _record_visit(p, cur["name"], from_name=None):
                                reached = True
                                agents[p]["active"] = False
                                break

                # α / β learning jump (teleport to remembered/best)
                if not reached and (random.random() < α):
                    target = None
                    if random.random() < β and len(phist.get(p, [])) > 1:
                        window = phist[p][-beta_memory:]
                        rec = random.choice([t.split("_",1)[1].rstrip("*") for t in window])
                        if rec in G.vs["name"]:
                            target = G.vs.find(name=rec)
                    if target is None and cache_votes:
                        best = max(cache_votes, key=cache_votes.get)
                        if best in G.vs["name"]:
                            target = G.vs.find(name=best)
                    if target is not None and target["name"] != cur["name"]:
                        cur = target
                        agents[p]["current"] = cur
                        if _record_visit(p, cur["name"], from_name=None):
                            reached = True
                            agents[p]["active"] = False
                            break

                if reached:
                    break  # break out of 'for p in active'

                # neighbors from current node
                nbrs = G.neighbors(cur.index, mode="out")
                neighbor_names = [G.vs[nb]["name"] for nb in nbrs]

                # optional filler inspections (do NOT change current; just log inspections if enabled)
                if include_filler_offers and neighbor_names:
                    all_fillers = get_filler_offers_for_anchor(cur["name"], nego_df, max_search_distance)
                    all_fillers = [f for f in all_fillers if f != cur["name"] and f not in neighbor_names]
                    if all_fillers:
                        k = random.randint(1, len(all_fillers))
                        for filler_dest in random.sample(all_fillers, k):
                            # filler inspections are recorded in path history as non-consensus "looks"
                            path.append(filler_dest)
                            ann.append(f"{p}_{filler_dest}")
                            ann_val.append((None, None))

                if not nbrs:
                    agents[p]["active"] = False
                    continue

                # apply transition rule to neighbors (safe lookups + no revisits)
                cand = []
                for nb in nbrs:
                    dest = G.vs[nb]["name"]

                    # never revisit by THIS agent
                    if dest in visited_nodes[p]:
                        continue

                    row = _edge_row(nego_df, cur["name"], dest)
                    if row is None:
                        continue  # edge missing in nego_df; skip

                    vote_chg = row.get("Vote_Change", 0)
                    dest_pay = row.get(f"{p}_destination_payoff", -float("inf"))
                    orig_pay = row.get(f"{p}_origin_payoff",    -float("inf"))

                    if transition_rule == 2:  # tie-resolution
                        if vote_chg > 0 or (vote_chg == 0 and dest_pay > orig_pay):
                            cand.append(nb)

                    elif transition_rule == 3:  # proposer-accept
                        proposer_ok = _party_accepts(p, row.get("Accepting", "None"))
                        if (vote_chg > 0 and proposer_ok) or (vote_chg == 0 and dest_pay > orig_pay):
                            cand.append(nb)

                    else:  # rules 1,4,5: keep strict vote gains only (safe)
                        if vote_chg > 0:
                            cand.append(nb)

                if not cand:
                    agents[p]["active"] = False
                    continue

                nxt = G.vs[random.choice(cand)]
                agents[p]["current"] = nxt
                visited_nodes[p].add(nxt["name"])

                # tag next step safely (edge-based move)
                rowd = _edge_row(nego_df, cur["name"], nxt["name"])
                dvotes = rowd.get("Destination_votes", None) if rowd is not None else None
                dpay   = rowd.get(f"{p}_destination_payoff", None) if rowd is not None else None
                tag  = f"{p}_{nxt['name']}*"
                tagv = f"{tag} ({dvotes}, {dpay})"

                # below-RP?
                if rowd is None or not _party_accepts(p, rowd.get("Accepting", "None")):
                    below_rp[p] += 1

                path.append(nxt["name"]); ann.append(tag); ann_val.append(tagv)
                phist.setdefault(p, []).append(tag); phist_val.setdefault(p, []).append(tagv)

                # parameter drift
                α = min(1, α + delta_alpha); β = min(1, β + delta_beta)
                γ = min(1, γ + delta_gamma); θ = min(1, θ + delta_theta)

                cache_votes.setdefault(nxt["name"], vote_max.get(nxt["name"], 0))

                # move-based consensus check (unchanged behavior)
                if nxt["name"] in consensus_nodes:
                    reached = True; agents[p]["active"] = False; break

        # -------- metrics & record --------
        if len(path) > 1:
            dists = [_hamming(path[i], path[i+1]) for i in range(len(path)-1)]
            dm, dv = sum(dists)/len(dists), pd.Series(dists).var()
        else:
            dm = dv = 0.0
        fl = _hamming(path[0], path[-1]) if path else 0

        out = dict(
            start_node          = path[0] if path else None,
            end_node            = path[-1] if path else None,
            first_offer_party   = first_offer,
            agreement           = int(reached),
            agreement_step      = len(path) if reached else None,
            search_steps        = len(path),
            # preparation adds one inspection per candidate per party
            search_steps_prep   = len(path) + top_value * len(parties),
            first_last_hamming  = fl,
            path_history        = path,
            stepdist_mean       = dm,
            stepdist_var        = dv,
        )

        if record_full_histories:
            out["annotated_path_history"]        = ann
            out["annotated_path_history_values"] = ann_val
            for q in parties:
                out[f"annotated_history_{q}"]     = phist_val.get(q, [])
                out[f"annotated_history_raw_{q}"] = phist.get(q, [])

        for q in parties:
            out[f"{q}_below_rp"] = below_rp[q]

        if add_impasse:
            cap = out["agreement_step"] or len(path)
            for i in range(1, 31):
                out[f"i_{i}"] = int(i < cap)

        runs.append(out)

    runs_df = pd.DataFrame(runs)

    # -------- optional post-processing (unchanged) --------
    if (not include_filler_offers) and apply_post_processing and record_full_histories:
        cache = {}
        def _m(n):
            if n not in cache:
                cache[n] = len(get_filler_offers_for_anchor(n, nego_df, max_search_distance))
            return cache[n]
        def _patch(r):
            extra = 0
            byp = {}
            for t in r["annotated_path_history"]:
                if t.endswith("*"):
                    q, anc = t.split("_",1); anc = anc.rstrip("*")
                    byp.setdefault(q, []).append(anc)
            for lst in byp.values():
                for a in lst[:-1]:
                    extra += random.randint(0, _m(a)-1) if _m(a) else 0
            return r["search_steps"] + extra
        runs_df["search_steps"] = runs_df.apply(_patch, axis=1)

    return runs_df



# ───────────────────── deadline helper (new, vectorized) ────────────────────
def _compute_deadline_rates_vectorized(runs_df: pd.DataFrame, Tvals) -> dict:
    """
    For thresholds T in Tvals, compute AgreeRate@T = P(agreement_step <= T).
    Returns dict {f"AgreeRate@{int(T)}": float}. Assumes 'agreement' and 'agreement_step' exist.
    """
    if runs_df is None or runs_df.empty:
        return {f"AgreeRate@{int(T)}": np.nan for T in Tvals}
    agre = pd.to_numeric(runs_df.get("agreement", np.nan), errors="coerce").fillna(0).astype(int).to_numpy()
    step = pd.to_numeric(runs_df.get("agreement_step", np.nan), errors="coerce").to_numpy()
    agree_steps = step[(agre == 1) & np.isfinite(step)]
    n = len(agre) if len(agre) else 1
    agree_steps.sort(kind="mergesort")
    out = {}
    for T in Tvals:
        rate = np.searchsorted(agree_steps, T, side="right") / n
        out[f"AgreeRate@{int(T)}"] = float(rate)
    return out


# ─────────────────────────── main grid driver ───────────────────────────────
# ─────────────────────────── main grid driver ───────────────────────────────
def main_search_model(
        *, csv_list, output_path, extra_csv_folder=None,
        learning_rate=0.0, learning_rate_delta=0,
        beta=0.0, beta_delta=0, beta_memory=1,
        gamma=0.0, gamma_delta=0, gamma_dist=1,
        theta=0.0, theta_delta=0, theta_override_alpha=False,
        max_search_distance=1,
        first_offer_rule="top_percent", top_value=3,
        agreement_rule=3, transition_rule=3,
        num_simulations=100, random_seed=None,
        include_filler_offers=False, apply_post_processing=False,
        add_impasse=False, record_full_histories=True,
        summarize_payoffs=False, reservation_callback=None,
        # NEW:
        add_deadline_columns=True,
        deadline_time_limits=tuple(range(0, 101, 2)),
):
    import itertools
    import glob
    import pandas as pd
    from tqdm import tqdm
    from pathlib import Path

    # collect scenarios --------------------------------------------------
    csvs = list(csv_list)
    if extra_csv_folder:
        csvs += sorted(glob.glob(os.path.join(extra_csv_folder, "*.csv")))

    to_lst = lambda x: x if isinstance(x, (list, tuple)) else [x]

    grids = dict(
        a0   = to_lst(learning_rate),        da   = to_lst(learning_rate_delta),
        b0   = to_lst(beta),                 db   = to_lst(beta_delta),
        g0   = to_lst(gamma),                dg   = to_lst(gamma_delta),
        t0   = to_lst(theta),                dt   = to_lst(theta_delta),
        dmax = to_lst(max_search_distance),
        fo   = to_lst(first_offer_rule),     tv   = to_lst(top_value),
        agr  = to_lst(agreement_rule),       trn  = to_lst(transition_rule),
    )

    if reservation_callback is None:
        reservation_callback = lambda n,p,df: pd.DataFrame({"Party": p, "Reservation": [0]*len(p)})

    # ---- Phase 1: fast setup (single-line; cleared when done) -----------
    resolved_paths = []
    setup_bar = tqdm(total=len(csvs), desc="Setup: scanning CSVs", unit="file",
                     dynamic_ncols=True, leave=False, position=0)
    for c in csvs:
        p = Path(c)
        rp = str(p.resolve()) if p.exists() else str(p)
        resolved_paths.append(rp)
        setup_bar.set_postfix_str(p.name)
        setup_bar.update(1)
    setup_bar.close()

    print_simulation_plan(resolved_paths, grids)

    # totals for progress bar --------------------------------------------
    num_scenarios = (len(csvs) * len(grids["fo"]) * len(grids["tv"]) *
                     len(grids["agr"]) * len(grids["trn"]) * len(grids["dmax"]) *
                     len(grids["a0"]) * len(grids["da"]) *
                     len(grids["b0"]) * len(grids["db"]) *
                     len(grids["g0"]) * len(grids["dg"]) *
                     len(grids["t0"]) * len(grids["dt"]))
    total = num_scenarios * num_simulations

    # ---- Phase 2: simulations (single line) ----------------------------
    bar = tqdm(total=total, desc="Simulations", unit="run",
               dynamic_ncols=True, leave=True, position=0)

    summary = []

    # Cartesian product over every grid dimension ------------------------
    for (csv_name, fo, tv, agr, trn, dmax,
         a0, da, b0, db, g0, dg, t0, dt) in itertools.product(
            csvs, grids["fo"], grids["tv"],
            grids["agr"], grids["trn"], grids["dmax"],
            grids["a0"], grids["da"], grids["b0"], grids["db"],
            grids["g0"], grids["dg"], grids["t0"], grids["dt"]):

        try_name = Path(str(csv_name)).name
        bar.set_postfix_str(f"{try_name} | fo={fo} tv={tv} agr={agr} trn={trn} dmax={dmax}")

        # robust read + pruned csv_source (real read happens here)
        df_raw, resolved_path = read_input_csv(csv_name, output_path)
        csv_base = sanitize_csv_source(resolved_path)
        scen_type = scenario_type_from_basename(csv_base)

        possible_solutions = compute_possible_solutions(df_raw)

        # ---------------- preprocessing ----------------
        max_votes = df_raw["Origin_votes"].max()
        df_raw = set_agreement_threshold(df_raw, max_votes, rule_choice=agr)
        df_raw["Reached Consensus"] = df_raw["Destination_Agreement"].eq(1)

        # consensus nodes: NO "0/2" filter
        cons_nodes = df_raw.loc[df_raw["Reached Consensus"], "Destination"].unique().tolist()
        ZOPA = len(cons_nodes)

        df_trans = apply_transition_rules_to_dataframe(df_raw, rule_number=trn)

        # ----------- first starting-anchors table -----------
        top_percent_flag = (fo == "top_percent")
        starts_base = generate_chosen_solution_info(
            df_raw,
            first_offer_rule      = fo,
            top_value             = tv,
            top_percent           = top_percent_flag,
            random_seed           = random_seed
        )

        parties = starts_base["Party"].unique().tolist()
        N_part  = len(parties)
        M_iss   = int(pd.Series(df_raw["Destination"].astype(str).unique()).str.len().mean())
        rp_df   = reservation_callback(csv_base, parties, df_raw)

        edges = df_trans[df_trans["Option_Change"] <= dmax]
        G     = ig.Graph.TupleList(
                    edges[["Origin", "Destination"]].itertuples(index=False),
                    directed=True, vertex_name_attr="name"
                )

        valid   = set(G.vs["name"])
        starts  = starts_base[starts_base["Proposal"].isin(valid)].copy()
        missing = set(parties) - set(starts["Party"])
        if missing:
            df_filt  = df_raw[df_raw["Origin"].isin(valid)].copy()
            add_back = generate_chosen_solution_info(
                df_filt,
                first_offer_rule      = fo,
                top_value             = tv,
                top_percent           = top_percent_flag,
                random_seed           = random_seed
            )
            starts = pd.concat([starts, add_back[add_back["Party"].isin(missing)]], ignore_index=True)

        if starts.empty:
            bar.update(num_simulations)
            continue

        # -------------- core engine call (now passes trn) ---------------
        runs_df = _run_simulation(
            G, starts, cons_nodes, df_raw,
            num_simulations       = num_simulations,
            first_offer_rule      = fo,
            top_value             = tv,
            alpha0                = a0,  delta_alpha = da,
            beta0                 = b0,  delta_beta  = db,  beta_memory = beta_memory,
            gamma0                = g0,  delta_gamma = dg,  gamma_dist   = gamma_dist,
            theta0                = t0,  delta_theta = dt,  theta_override_alpha = theta_override_alpha,
            include_filler_offers = include_filler_offers,
            max_search_distance   = dmax,
            random_seed           = random_seed,
            add_impasse           = add_impasse,
            apply_post_processing = apply_post_processing,
            record_full_histories = record_full_histories,
            transition_rule       = trn
        )

        _attach_final_payoffs(runs_df, df_raw, rp_df)
        runs_df["possible_solutions"] = possible_solutions
        runs_df["Exploration_Rate"]   = runs_df["path_history"].apply(
            lambda path: compute_exploration_rate(path, possible_solutions)
        )

        # >>> NEW: per-run First-Offer Advantage indicator for each party <<<
        # For each party, compare payoff when they are first vs when not first,
        # using within-scenario empirical means for the counterfactual.
        for party in parties:
            pay_col = f"{party}_Final_Payoff"
            mask_first = runs_df["first_offer_party"] == party
            mean_when_first    = runs_df.loc[mask_first,  pay_col].mean() if mask_first.any() else np.nan
            mean_when_notfirst = runs_df.loc[~mask_first, pay_col].mean() if (~mask_first).any() else np.nan

            ind_vals = []
            for _, row_r in runs_df.iterrows():
                pay = row_r.get(pay_col, np.nan)
                if pd.isna(pay) or (pd.isna(mean_when_first) and pd.isna(mean_when_notfirst)):
                    ind_vals.append(np.nan)
                    continue

                if row_r["first_offer_party"] == party:
                    comp = mean_when_notfirst
                    if pd.isna(comp):
                        ind_vals.append(np.nan)
                    else:
                        if pay > comp:
                            ind_vals.append(1.0)
                        elif np.isclose(pay, comp):
                            ind_vals.append(0.5)
                        else:
                            ind_vals.append(0.0)
                else:
                    comp = mean_when_first
                    if pd.isna(comp):
                        ind_vals.append(np.nan)
                    else:
                        if comp > pay:
                            ind_vals.append(1.0)
                        elif np.isclose(comp, pay):
                            ind_vals.append(0.5)
                        else:
                            ind_vals.append(0.0)

            runs_df[f"{party}_FirstOfferAdv_Ind"] = ind_vals
        # <<< END NEW >>>

        # reflect run progress on the single bar
        bar.update(len(runs_df))

        # --------------------- summarise & store ------------------------
        base_code = make_simulation_code(
            csv_base, a0=a0, da=da, b0=b0, db=db, g0=g0, dg=dg, t0=t0, dt=dt,
            dmax=dmax, agr=agr, trn=trn, fo=fo
        )
        code = ensure_unique_key(SIMULATION_STORE, base_code)
        SIMULATION_STORE[code] = runs_df

        # NEW: compute deadlines once per scenario, vectorized from runs_df
        deadline_cols = {}
        if add_deadline_columns and deadline_time_limits:
            Tvals = [int(t) for t in deadline_time_limits]
            deadline_cols = _compute_deadline_rates_vectorized(runs_df, Tvals)

        row = dict(
            simulation_code               = code,
            csv_source                    = csv_base,
            S2_OUTPUT_folder              = str(output_path),  # handy for later joins
            Scenario_Type                 = scen_type,
            possible_solutions            = possible_solutions,
            ZOPA                          = ZOPA,
            ZOPA_Pct                      = round(ZOPA / possible_solutions, 4),
            N                             = N_part,
            M                             = M_iss,
            Agreement_Rate                = round(runs_df["agreement"].mean(), 2),
            Exploration_Rate              = round(runs_df["Exploration_Rate"].mean(), 4),
            Mean_Offer_Count              = round(runs_df["search_steps"].mean(), 2),
            first_offer_rule              = fo,
            top_value                     = tv,
            agreement_rule                = agr,
            transition_rule               = trn,
            alpha                         = a0,
            delta_alpha                   = da,
            beta                          = b0,
            delta_beta                    = db,
            beta_memory                   = beta_memory,
            gamma                         = g0,
            delta_gamma                   = dg,
            gamma_dist                    = gamma_dist,
            theta                         = t0,
            delta_theta                   = dt,
            theta_override_alpha          = theta_override_alpha,
            max_search_distance           = dmax,
            Runs                          = len(runs_df),
            Mean_Offer_Count_Prep         = round(runs_df["search_steps_prep"].mean(), 2),
            Mean_Offer_Count_Agree        = round(runs_df.loc[runs_df.agreement==1, "search_steps"].mean(), 2),
            Mean_Offer_Count_NoAgree      = round(runs_df.loc[runs_df.agreement==0, "search_steps"].mean(), 2),
            Mean_Offer_Distance           = round(runs_df["stepdist_mean"].mean(), 2),
            Var_Offer_Distance            = round(runs_df["stepdist_var"].mean(), 2),
            Mean_FirstLast_Distance       = round(runs_df["first_last_hamming"].mean(), 2),
            Mean_FirstLast_Distance_Agree = round(runs_df.loc[runs_df.agreement==1, "first_last_hamming"].mean(), 2),
            Mean_FirstLast_Distance_NoAgree=round(runs_df.loc[runs_df.agreement==0, "first_last_hamming"].mean(), 2),
        )

        # attach deadline columns to summary row
        if deadline_cols:
            row.update({k: round(v, 4) for k, v in deadline_cols.items()})

        if summarize_payoffs:
            fl_series = runs_df["first_last_hamming"]
            for i, party in enumerate(parties, 1):
                pay   = runs_df[f"{party}_Final_Payoff"]
                first = runs_df["first_offer_party"] == party
                agree = runs_df["agreement"] == 1

                row[f"P{i}_FirstOffer"]       = int(first.sum())
                row[f"P{i}_NotFirstOffer"]    = len(runs_df) - first.sum()
                row[f"P{i}_PayAll"]           = round(pay.mean(), 2)
                row[f"P{i}_PayAgree"]         = round(pay[agree].mean(), 2)
                row[f"P{i}_PayNoAgree"]       = round(pay[~agree].mean(), 2)
                row[f"P{i}_PayFirstOffer"]    = round(pay[first].mean(), 2)
                row[f"P{i}_PayNotFirstOffer"] = round(pay[~first].mean(), 2)
                row[f"P{i}_PayFirstOfferAgree"]    = round(pay[first & agree].mean(), 2)
                row[f"P{i}_PayFirstOfferNoAgree"]  = round(pay[first & ~agree].mean(), 2)
                row[f"P{i}_PayNotFirstOfferAgree"] = round(pay[~first & agree].mean(), 2)
                row[f"P{i}_PayNotFirstOfferNoAgree"]= round(pay[~first & ~agree].mean(), 2)
                row[f"P{i}_AgreeWhenFirst"]    = round(agree[first].mean(), 2)
                row[f"P{i}_AgreeWhenNotFirst"] = round(agree[~first].mean(), 2)
                row[f"P{i}_FLDistFirst"]       = round(fl_series[first].mean(), 2)
                row[f"P{i}_FLDistNotFirst"]    = round(fl_series[~first].mean(), 2)
                row[f"P{i}_FLDistFirstAgree"]  = round(fl_series[first & agree].mean(), 2)
                row[f"P{i}_FLDistFirstNoAgree"]= round(fl_series[first & ~agree].mean(), 2)

                # >>> NEW: summary probability of first-offer advantage <<<
                adv_col = f"{party}_FirstOfferAdv_Ind"
                row[f"P{i}_FirstOfferAdv_Prob"] = round(runs_df[adv_col].mean(), 4)
                # <<< END NEW >>>

        summary.append(row)

    bar.close()
    return pd.DataFrame(summary)



### Run code for simulation

In [ ]:
# ── PARAMETERS ──────────────────────────────────────────────────────────────
num_simulations = 1000
random_seed     = 1

# alpha grid (example)
learning_rate        = [0.75]
learning_rate_delta  = [0]

# First offer rules
first_offer_rule     = ['random']   # could also include ['meso','random']
top_value            = [3]

# Search rules
agreement_rule       = [3]
transition_rule      = [1]
max_search_distance  = [2]

# Toggles
include_filler_offers = True
apply_post_processing = False
record_full_histories = True
summarize_payoffs     = True
add_deadline          = True

# Noise parameters
beta                 = [0]
beta_delta           = [0]
beta_memory          = 1
gamma                = [0]
gamma_delta          = [0]
gamma_dist           = 1
theta                = [0]
theta_delta          = [0]
theta_override_alpha = True

# ── MANUAL / GENERIC MODE ───────────────────────────────────────────────────
run_mode = "generic"

# Generic mode choices
N_filters            = [2]     # ← filter by group size N (set to None to ignore)
M_filters            = [7]     # ← NEW: filter by issue count M (set to None to ignore)
Manual_limit_per_N   = 2000   # optional cap per N (still applies after N/M filtering)

##############################################################################
csv_list = []
extra_csv_folder = None

if run_mode in ("manual", "both"):
    csv_file_names = [
        "FL_old_transition_analysis.csv",
        "FL_new_transition_analysis.csv",
        "harborco_transition_analysis.csv",
        "new_recruit_mod_transition_analysis.csv",
    ]
    csv_list_manual = [str(S1_OUTPUT / name) for name in csv_file_names]
else:
    csv_list_manual = []

if run_mode in ("generic", "both"):
    # Grab all candidate CSVs from S1 output
    all_csvs = sorted(S1_OUTPUT.glob("*.csv"))
    csv_list_generic = []

    # Helper: decide if a filename matches the N/M filters
    # Files look like "UM_N=6_M=7_999.csv" (your convention)
    import re
    pat = re.compile(r"UM_N=(\d+)_M=(\d+)_")

    def match_nm(pathname, Ns, Ms):
        m = pat.search(pathname)
        if not m:
            return False
        n_val = int(m.group(1))
        m_val = int(m.group(2))
        n_ok = (Ns is None) or (len(Ns) == 0) or (n_val in Ns)
        m_ok = (Ms is None) or (len(Ms) == 0) or (m_val in Ms)
        return n_ok and m_ok

    # Build list based on provided filters
    selected = [f for f in all_csvs if match_nm(f.name, N_filters, M_filters)]

    # Optionally cap per-N after filtering by M as well
    if Manual_limit_per_N:
        # bucket by N, then take first k for each N
        from collections import defaultdict
        buckets = defaultdict(list)
        for f in selected:
            m = pat.search(f.name)
            if not m:
                continue
            n_val = int(m.group(1))
            buckets[n_val].append(f)
        capped = []
        for n_val, files in buckets.items():
            files = sorted(files)
            capped.extend(files[:Manual_limit_per_N])
        selected = capped

    csv_list_generic = [str(f) for f in selected]
else:
    csv_list_generic = []

if run_mode == "manual":
    csv_list = csv_list_manual
elif run_mode == "generic":
    csv_list = csv_list_generic
elif run_mode == "both":
    seen = set()
    csv_list = []
    for p in csv_list_manual + csv_list_generic:
        if p not in seen:
            seen.add(p)
            csv_list.append(p)
else:
    raise ValueError("run_mode must be 'manual', 'generic', or 'both'.")

# Deadlines we want baked into the summary (same as your analysis defaults)
DEADLINE_TIME_LIMITS = list(range(0, 101, 2))  # 0,2,4,...,100

# ── RUN ─────────────────────────────────────────────────────────────────────
summary_df_adv = main_search_model(
    csv_list=csv_list,
    extra_csv_folder=extra_csv_folder,
    output_path=S2_OUTPUT,
    num_simulations=num_simulations,
    learning_rate=learning_rate,
    learning_rate_delta=learning_rate_delta,
    max_search_distance=max_search_distance,
    first_offer_rule=first_offer_rule,
    top_value=top_value,
    agreement_rule=agreement_rule,
    transition_rule=transition_rule,
    random_seed=random_seed,
    include_filler_offers=include_filler_offers,
    apply_post_processing=apply_post_processing,
    record_full_histories=record_full_histories,
    summarize_payoffs=summarize_payoffs,
    add_deadline_columns=add_deadline,
    deadline_time_limits=DEADLINE_TIME_LIMITS,
)

# ── SAVE ────────────────────────────────────────────────────────────────────
save_results(summary_df_adv, dest_folder=S2_OUTPUT)

# PRINT
summary_df_adv



=== Simulation Plan ===
CSV sources   : UM_N=2_M=7_1.csv, UM_N=2_M=7_10.csv, UM_N=2_M=7_100.csv, UM_N=2_M=7_1000.csv, UM_N=2_M=7_1001.csv, UM_N=2_M=7_1002.csv, UM_N=2_M=7_1003.csv, UM_N=2_M=7_1004.csv, UM_N=2_M=7_1005.csv, UM_N=2_M=7_1006.csv, UM_N=2_M=7_1007.csv, UM_N=2_M=7_1008.csv, UM_N=2_M=7_1009.csv, UM_N=2_M=7_101.csv, UM_N=2_M=7_1010.csv, UM_N=2_M=7_1011.csv, UM_N=2_M=7_1012.csv, UM_N=2_M=7_1013.csv, UM_N=2_M=7_1014.csv, UM_N=2_M=7_1015.csv, UM_N=2_M=7_1016.csv, UM_N=2_M=7_1017.csv, UM_N=2_M=7_1018.csv, UM_N=2_M=7_1019.csv, UM_N=2_M=7_102.csv, UM_N=2_M=7_1020.csv, UM_N=2_M=7_1021.csv, UM_N=2_M=7_1022.csv, UM_N=2_M=7_1023.csv, UM_N=2_M=7_1024.csv, UM_N=2_M=7_1025.csv, UM_N=2_M=7_1026.csv, UM_N=2_M=7_1027.csv, UM_N=2_M=7_1028.csv, UM_N=2_M=7_1029.csv, UM_N=2_M=7_103.csv, UM_N=2_M=7_1030.csv, UM_N=2_M=7_1031.csv, UM_N=2_M=7_1032.csv, UM_N=2_M=7_1033.csv, UM_N=2_M=7_1034.csv, UM_N=2_M=7_1035.csv, UM_N=2_M=7_1036.csv, UM_N=2_M=7_1037.csv, UM_N=2_M=7_1038.csv, UM_N=2_M=7_1039.csv, UM

Simulations: 100%|██████████| 2000000/2000000 [2:56:22<00:00, 189.00run/s, UM_N=2_M=7_999.csv | fo=random tv=3 agr=3 trn=1 dmax=2]


Saved results to /content/drive/MyDrive/Negotiations-as-Search/S2_OUTPUT/simulation_20250928_144003


,simulation_code,csv_source,S2_OUTPUT_folder,Scenario_Type,possible_solutions,ZOPA,ZOPA_Pct,N,M,Agreement_Rate,...,P2_PayFirstOfferNoAgree,P2_PayNotFirstOfferAgree,P2_PayNotFirstOfferNoAgree,P2_AgreeWhenFirst,P2_AgreeWhenNotFirst,P2_FLDistFirst,P2_FLDistNotFirst,P2_FLDistFirstAgree,P2_FLDistFirstNoAgree,P2_FirstOfferAdv_Prob
0,UM_N=2_M=7_1__tr1_S2_fo_random_a0.75_b0.0_g0.0...,UM_N=2_M=7_1.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,18,0.1406,2,7,0.91,...,0.0,0.71,0.0,0.91,0.92,1.91,1.93,2.05,0.59,0.482
1,UM_N=2_M=7_10__tr1_S2_fo_random_a0.75_b0.0_g0....,UM_N=2_M=7_10.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,35,0.2734,2,7,0.96,...,0.0,0.79,0.0,0.96,0.97,1.92,1.96,1.99,0.59,0.491
2,UM_N=2_M=7_100__tr1_S2_fo_random_a0.75_b0.0_g0...,UM_N=2_M=7_100.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,14,0.1094,2,7,0.91,...,0.0,0.37,0.0,0.92,0.91,2.25,2.35,2.40,0.60,0.495
3,UM_N=2_M=7_1000__tr1_S2_fo_random_a0.75_b0.0_g...,UM_N=2_M=7_1000.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,9,0.0703,2,7,0.97,...,0.0,0.23,0.0,0.97,0.97,2.72,2.67,2.74,1.94,0.507
4,UM_N=2_M=7_1001__tr1_S2_fo_random_a0.75_b0.0_g...,UM_N=2_M=7_1001.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,38,0.2969,2,7,0.98,...,0.0,0.95,0.0,0.98,0.98,1.79,1.88,1.82,0.56,0.474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,UM_N=2_M=7_995__tr1_S2_fo_random_a0.75_b0.0_g0...,UM_N=2_M=7_995.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,14,0.1094,2,7,0.91,...,0.0,0.68,0.0,0.92,0.91,2.34,2.30,2.44,1.10,0.571
1996,UM_N=2_M=7_996__tr1_S2_fo_random_a0.75_b0.0_g0...,UM_N=2_M=7_996.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,26,0.2031,2,7,0.97,...,0.0,0.74,0.0,0.97,0.96,2.31,2.10,2.34,1.08,0.490
1997,UM_N=2_M=7_997__tr1_S2_fo_random_a0.75_b0.0_g0...,UM_N=2_M=7_997.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,9,0.0703,2,7,0.87,...,0.0,0.13,0.0,0.86,0.87,2.44,2.33,2.61,1.38,0.484
1998,UM_N=2_M=7_998__tr1_S2_fo_random_a0.75_b0.0_g0...,UM_N=2_M=7_998.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,67,0.5234,2,7,1.00,...,NaN,0.58,NaN,1.00,1.00,1.53,1.43,1.53,NaN,0.479


In [ ]:
# ── PARAMETERS ──────────────────────────────────────────────────────────────
num_simulations = 1000
random_seed     = 1

# alpha grid (example)
learning_rate        = [0.25, 0.75]
learning_rate_delta  = [0]

# First offer rules
first_offer_rule     = ['random']   # could also include ['meso','random']
top_value            = [3]

# Search rules
agreement_rule       = [3]
transition_rule      = [2]
max_search_distance  = [1]

# Toggles
include_filler_offers = True
apply_post_processing = False
record_full_histories = True
summarize_payoffs     = True
add_deadline          = True

# Noise parameters
beta                 = [0]
beta_delta           = [0]
beta_memory          = 1
gamma                = [0]
gamma_delta          = [0]
gamma_dist           = 1
theta                = [0]
theta_delta          = [0]
theta_override_alpha = True

# ── MANUAL / GENERIC MODE ───────────────────────────────────────────────────
run_mode = "generic"

# Generic mode choices
N_filters            = [2]     # ← filter by group size N (set to None to ignore)
M_filters            = [7]     # ← NEW: filter by issue count M (set to None to ignore)
Manual_limit_per_N   = 2000    # optional cap per N (still applies after N/M filtering)

##############################################################################
csv_list = []
extra_csv_folder = None

if run_mode in ("manual", "both"):
    csv_file_names = [
        "FL_old_transition_analysis.csv",
        "FL_new_transition_analysis.csv",
        "harborco_transition_analysis.csv",
        "new_recruit_mod_transition_analysis.csv",
    ]
    csv_list_manual = [str(S1_OUTPUT / name) for name in csv_file_names]
else:
    csv_list_manual = []

if run_mode in ("generic", "both"):
    # Grab all candidate CSVs from S1 output
    all_csvs = sorted(S1_OUTPUT.glob("*.csv"))
    csv_list_generic = []

    # Helper: decide if a filename matches the N/M filters
    # Files look like "UM_N=6_M=7_999.csv" (your convention)
    import re
    pat = re.compile(r"UM_N=(\d+)_M=(\d+)_")

    def match_nm(pathname, Ns, Ms):
        m = pat.search(pathname)
        if not m:
            return False
        n_val = int(m.group(1))
        m_val = int(m.group(2))
        n_ok = (Ns is None) or (len(Ns) == 0) or (n_val in Ns)
        m_ok = (Ms is None) or (len(Ms) == 0) or (m_val in Ms)
        return n_ok and m_ok

    # Build list based on provided filters
    selected = [f for f in all_csvs if match_nm(f.name, N_filters, M_filters)]

    # Optionally cap per-N after filtering by M as well
    if Manual_limit_per_N:
        # bucket by N, then take first k for each N
        from collections import defaultdict
        buckets = defaultdict(list)
        for f in selected:
            m = pat.search(f.name)
            if not m:
                continue
            n_val = int(m.group(1))
            buckets[n_val].append(f)
        capped = []
        for n_val, files in buckets.items():
            files = sorted(files)
            capped.extend(files[:Manual_limit_per_N])
        selected = capped

    csv_list_generic = [str(f) for f in selected]
else:
    csv_list_generic = []

if run_mode == "manual":
    csv_list = csv_list_manual
elif run_mode == "generic":
    csv_list = csv_list_generic
elif run_mode == "both":
    seen = set()
    csv_list = []
    for p in csv_list_manual + csv_list_generic:
        if p not in seen:
            seen.add(p)
            csv_list.append(p)
else:
    raise ValueError("run_mode must be 'manual', 'generic', or 'both'.")

# Deadlines we want baked into the summary (same as your analysis defaults)
DEADLINE_TIME_LIMITS = list(range(0, 101, 2))  # 0,2,4,...,100

# ── RUN ─────────────────────────────────────────────────────────────────────
summary_df_adv = main_search_model(
    csv_list=csv_list,
    extra_csv_folder=extra_csv_folder,
    output_path=S2_OUTPUT,
    num_simulations=num_simulations,
    learning_rate=learning_rate,
    learning_rate_delta=learning_rate_delta,
    max_search_distance=max_search_distance,
    first_offer_rule=first_offer_rule,
    top_value=top_value,
    agreement_rule=agreement_rule,
    transition_rule=transition_rule,
    random_seed=random_seed,
    include_filler_offers=include_filler_offers,
    apply_post_processing=apply_post_processing,
    record_full_histories=record_full_histories,
    summarize_payoffs=summarize_payoffs,
    add_deadline_columns=add_deadline,
    deadline_time_limits=DEADLINE_TIME_LIMITS,
)

# ── SAVE ────────────────────────────────────────────────────────────────────
save_results(summary_df_adv, dest_folder=S2_OUTPUT)

# PRINT
summary_df_adv



=== Simulation Plan ===
CSV sources   : UM_N=2_M=7_1.csv, UM_N=2_M=7_10.csv, UM_N=2_M=7_100.csv, UM_N=2_M=7_1000.csv, UM_N=2_M=7_1001.csv, UM_N=2_M=7_1002.csv, UM_N=2_M=7_1003.csv, UM_N=2_M=7_1004.csv, UM_N=2_M=7_1005.csv, UM_N=2_M=7_1006.csv, UM_N=2_M=7_1007.csv, UM_N=2_M=7_1008.csv, UM_N=2_M=7_1009.csv, UM_N=2_M=7_101.csv, UM_N=2_M=7_1010.csv, UM_N=2_M=7_1011.csv, UM_N=2_M=7_1012.csv, UM_N=2_M=7_1013.csv, UM_N=2_M=7_1014.csv, UM_N=2_M=7_1015.csv, UM_N=2_M=7_1016.csv, UM_N=2_M=7_1017.csv, UM_N=2_M=7_1018.csv, UM_N=2_M=7_1019.csv, UM_N=2_M=7_102.csv, UM_N=2_M=7_1020.csv, UM_N=2_M=7_1021.csv, UM_N=2_M=7_1022.csv, UM_N=2_M=7_1023.csv, UM_N=2_M=7_1024.csv, UM_N=2_M=7_1025.csv, UM_N=2_M=7_1026.csv, UM_N=2_M=7_1027.csv, UM_N=2_M=7_1028.csv, UM_N=2_M=7_1029.csv, UM_N=2_M=7_103.csv, UM_N=2_M=7_1030.csv, UM_N=2_M=7_1031.csv, UM_N=2_M=7_1032.csv, UM_N=2_M=7_1033.csv, UM_N=2_M=7_1034.csv, UM_N=2_M=7_1035.csv, UM_N=2_M=7_1036.csv, UM_N=2_M=7_1037.csv, UM_N=2_M=7_1038.csv, UM_N=2_M=7_1039.csv, UM

Simulations: 100%|██████████| 4000000/4000000 [8:20:23<00:00, 133.23run/s, UM_N=2_M=7_999.csv | fo=random tv=3 agr=3 trn=2 dmax=1]


Saved results to /content/drive/MyDrive/Negotiations-as-Search/S2_OUTPUT/simulation_20250922_111115


,simulation_code,csv_source,S2_OUTPUT_folder,Scenario_Type,possible_solutions,ZOPA,ZOPA_Pct,N,M,Agreement_Rate,...,P2_PayFirstOfferNoAgree,P2_PayNotFirstOfferAgree,P2_PayNotFirstOfferNoAgree,P2_AgreeWhenFirst,P2_AgreeWhenNotFirst,P2_FLDistFirst,P2_FLDistNotFirst,P2_FLDistFirstAgree,P2_FLDistFirstNoAgree,P2_FirstOfferAdv_Prob
0,UM_N=2_M=7_1__tr2_S1_fo_random_a0.25_b0.0_g0.0...,UM_N=2_M=7_1.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,18,0.1406,2,7,1.00,...,0.0,0.81,0.0,1.00,1.00,2.51,2.45,2.52,1.00,0.492
1,UM_N=2_M=7_1__tr2_S1_fo_random_a0.75_b0.0_g0.0...,UM_N=2_M=7_1.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,18,0.1406,2,7,0.77,...,0.0,0.80,0.0,0.76,0.79,1.74,1.81,1.91,1.22,0.458
2,UM_N=2_M=7_10__tr2_S1_fo_random_a0.25_b0.0_g0....,UM_N=2_M=7_10.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,35,0.2734,2,7,1.00,...,0.0,0.82,NaN,1.00,1.00,2.01,2.13,2.01,1.00,0.511
3,UM_N=2_M=7_10__tr2_S1_fo_random_a0.75_b0.0_g0....,UM_N=2_M=7_10.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,35,0.2734,2,7,0.93,...,0.0,0.80,0.0,0.92,0.94,1.72,1.84,1.73,1.59,0.482
4,UM_N=2_M=7_100__tr2_S1_fo_random_a0.25_b0.0_g0...,UM_N=2_M=7_100.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,14,0.1094,2,7,0.99,...,0.0,0.40,0.0,0.99,0.99,2.56,2.49,2.56,2.33,0.470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,UM_N=2_M=7_997__tr2_S1_fo_random_a0.75_b0.0_g0...,UM_N=2_M=7_997.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,9,0.0703,2,7,0.61,...,0.0,0.15,0.0,0.62,0.60,1.96,2.10,2.16,1.64,0.508
3996,UM_N=2_M=7_998__tr2_S1_fo_random_a0.25_b0.0_g0...,UM_N=2_M=7_998.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,67,0.5234,2,7,1.00,...,NaN,0.49,NaN,1.00,1.00,1.43,1.41,1.43,NaN,0.541
3997,UM_N=2_M=7_998__tr2_S1_fo_random_a0.75_b0.0_g0...,UM_N=2_M=7_998.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,67,0.5234,2,7,0.99,...,0.0,0.49,0.0,0.99,1.00,1.30,1.33,1.30,1.40,0.548
3998,UM_N=2_M=7_999__tr2_S1_fo_random_a0.25_b0.0_g0...,UM_N=2_M=7_999.csv,/content/drive/MyDrive/Negotiations-as-Search/...,RanGen,128,19,0.1484,2,7,1.00,...,NaN,0.29,NaN,1.00,1.00,2.33,2.43,2.33,NaN,0.502


#### Bookmark: View latest progress of simulation
Scroll up from here to see simulation progress